In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import glob
import seaborn as sns

## Read from `~/metrics/epoch_metrics/` directory

In [2]:
df = pd.read_parquet("../../metrics/epoch_metrics/")

# Hyper-Parameters

## All tests summary

Note that this is a summary. 

Notes:
1. `synchronous` method implies `theta` equal to `0.0` and the reverse.
2. The stopping criterion (epochs at the moment) is not the same for all combination, in fact, it changes. For `LeNet-5` currently all tests end at 50 epochs. For the `AdvancedCNN` we go up to 350 epochs in some tests and some others up until 250.
3. `Theta` greater than 2 are exploratory tests and will not be considered at all later on.

In [3]:
for col in ['dataset_name', 'fda_name', 'nn_num_weights', 'num_clients', 'batch_size', 'num_steps_until_rtc_check', 'theta']:
    print(f"{col}: {sorted(list(df[col].unique()))}")

dataset_name: ['MNIST']
fda_name: ['linear', 'naive', 'sketch', 'synchronous']
nn_num_weights: [61706, 2592202]
num_clients: [5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60]
batch_size: [32, 64, 128, 256]
num_steps_until_rtc_check: [1]
theta: [0.0, 0.5, 1.0, 1.5, 2.0, 3.0, 5.0, 7.0, 10.0, 15.0, 20.0, 30.0, 50.0, 100.0]


## Query All tests

In [4]:
test_combinations = df.groupby(['dataset_name', 'nn_num_weights', 'fda_name', 'num_steps_until_rtc_check', 'batch_size', 'theta', 'num_clients'])['epoch'].max().reset_index()

In [5]:
test_combinations[
    (test_combinations['nn_num_weights'] == 61706) &
    (test_combinations['fda_name'] == 'sketch') &
    (test_combinations['theta'] == 2) &
    (test_combinations['num_clients'] == 5)
]

,dataset_name,nn_num_weights,fda_name,num_steps_until_rtc_check,batch_size,theta,num_clients,epoch


# Helpful new Dataframe metrics

### Add Helpful Dataset Metrics 

In [6]:
def dataset_n_train(row):
    if row['dataset_name'] == "MNIST":
        return 60_000
    else:
        return -1


def dataset_one_sample_bytes(row):
    if row['dataset_name'] == "MNIST":
        # input image 784 tf.float32 pixels and a tf.int32 label
        return 4 * (784 + 1)
    else:
        return -1

In [7]:
df['n_train'] = df.apply(dataset_n_train, axis=1)
df['one_sample_bytes'] = df.apply(dataset_one_sample_bytes, axis=1)

### Add Helpful model metrics

In [8]:
df['model_bytes'] = df['nn_num_weights'] * 4

### Add Helpful FDA method metrics

In [9]:
def fda_local_state_bytes(row):
    if row['fda_name'] == "naive":
        return 4
    if row['fda_name'] == "linear":
        return 8
    if row['fda_name'] == "sketch":
        return row['sketch_width'] * row['sketch_depth'] * 4 + 4
    if row['fda_name'] == "synchronous":
        return 0

In [10]:
df['local_state_bytes'] = df.apply(fda_local_state_bytes, axis=1)

### Add Total Steps

total steps (a single fda step might have many normal SGD steps, batch steps)

In [11]:
df['total_steps'] = df['total_fda_steps'] * df['num_steps_until_rtc_check']

### Add communication metrics

The communication bytes exchanged for model synchronization. Remember that the Clients send their models to the Server and the Server sends the global model back. This happens at the end of every round.

In [12]:
df['model_bytes_exchanged'] = df['total_rounds'] * df['model_bytes'] * df['num_clients'] * 2

The communication bytes exchanged for monitoring the variance. This happens at the end of every FDA step which consists of `num_steps_until_rtc_check` number of steps. 

In [13]:
df['monitoring_bytes_exchanged'] = df['local_state_bytes'] * df['total_fda_steps'] * df['num_clients']

The total communication bytes exchanged in the whole Federated Learning lifecycle.

In [14]:
df['total_communication_bytes'] = df['model_bytes_exchanged'] + df['monitoring_bytes_exchanged']

In [15]:
df['total_communication_gb'] = df['total_communication_bytes'] / 10**9

Add rounds in one epoch.

In [16]:
df = df.sort_values(by=['dataset_name', 'fda_name', 'nn_num_weights', 'num_clients', 'batch_size', 'num_steps_until_rtc_check', 'theta', 'epoch'])

df['epoch_rounds'] = df.groupby(['dataset_name', 'fda_name', 'nn_num_weights', 'num_clients', 'batch_size', 'num_steps_until_rtc_check', 'theta'])['total_rounds'].diff()

# NaN first epoch
df['epoch_rounds'] = df['epoch_rounds'].fillna(df['total_rounds'])

df['epoch_rounds'] = df['epoch_rounds'].astype(int)

# HyperParameter ranking

### LeNet-5
On 2 `Nvidia A10`:
1. Batch Size = 32 -> 6.613 s ± 0.128 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
2. *Batch Size* = 64 -> 7.509 s ± 0.065 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
3. *Batch Size* = 128 -> 8.02 s ± 0.099 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
4. *Batch Size* = 256 -> 9.258 s ± 0.336 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

## AdvancedCNN

On 2 `Nvidia A10`:
1. *Batch Size* = 32 -> 8.853 s ± 0.0917 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
2. *Batch Size* = 64 -> 10.325 ms ± 0.215 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
3. *Batch Size* = 128 -> 11.989 ms ± 0.134 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
4. *Batch Size* = 256 -> 16.47 ms ± 0.294 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

In [17]:
step_ms = {
    ("AdvancedCNN", 32): 8.853,
    ("AdvancedCNN", 64): 10.325,
    ("AdvancedCNN", 128): 11.989,
    ("AdvancedCNN", 256): 16.47,
    ("LeNet-5", 32): 6.613,
    ("LeNet-5", 64): 7.509,
    ("LeNet-5", 128): 8.02,
    ("LeNet-5", 256): 9.258
}

In [18]:
import numpy as np

def cpu_time_cost(row):
    """ Total cpu time cost in (sec).
    A single `step` means each client performed a single `step` 
    """
    return row['total_steps'] * step_ms[(row['nn_name'], row['batch_size'])] / 1000

def communication_time_cost(num_clients, total_communication_bytes, comm_model):
    """ Assuming channel is 1Gbps """

    total_communication_gbit = total_communication_bytes * 8e-9

    if comm_model == 'common_channel':
        
        return ((num_clients - 1) / num_clients) * total_communication_gbit    # sec

    if comm_model == 'hypercube':

        return (np.ceil(np.log(num_clients)) / num_clients) * total_communication_gbit   # sec

In [19]:
df['cpu_time_cost'] = df.apply(cpu_time_cost, axis=1)

In [20]:
df['hypercube_communication_time_cost'] = communication_time_cost(df['num_clients'], df['total_communication_bytes'], 'hypercube')

In [21]:
df['common_channel_communication_time_cost'] = communication_time_cost(df['num_clients'], df['total_communication_bytes'], 'common_channel')

In [22]:
df['hypercube_time_cost'] = df['cpu_time_cost'] + df['hypercube_communication_time_cost']

In [23]:
df['common_channel_time_cost'] = df['cpu_time_cost'] + df['common_channel_communication_time_cost']

In [24]:
df['hypercube_comm_cpu_time_ratio'] = df['hypercube_communication_time_cost'] / df['cpu_time_cost']

In [25]:
df['common_channel_comm_cpu_time_ratio'] = df['common_channel_communication_time_cost'] / df['cpu_time_cost']

# Plots about cost

In [26]:
# Define styles for each fda_name
fda_styles = {
    'naive': 'o-r',
    'linear': 's-g',
    'sketch': '^-b',
    'synchronous': 'x-c'
}
fda_names = sorted(df['fda_name'].unique())

In [27]:
import matplotlib

num_clients_values = sorted(df['num_clients'].unique())
cmap = matplotlib.colormaps['tab20b']
colors_dict = {
    num_clients: color 
    for num_clients, color in zip(num_clients_values, cmap(np.linspace(0, 1, len(num_clients_values))))
}

## Total time cost with accuracy (scatter)

### KDE

In [28]:
def kde_time_cost(df, filename):
    pdf = PdfPages(filename)
    
    fig, axs = plt.subplots(1, 2, figsize=(20, 6))
    
    hist_data_common_channel = []
    hist_data_hypercube = []

    # Prepare a list to store the average communication for each method
    avg_time_cost_common_channel_dict = {}
    avg_time_cost_hypercube_dict = {}
    
    # Prepare lists to store the average information (string) for each subplot
    avg_info_common_channel = []
    avg_info_hypercube = []

    for fda_name in fda_names:
        common_channel_df = df[df['fda_name'] == fda_name]['common_channel_time_cost']
        avg_time_cost_common_channel_dict[fda_name] = common_channel_df.mean()
        avg_info_common_channel.append(f'{fda_name}: {avg_time_cost_common_channel_dict[fda_name]:.2f} sec')
        sns.kdeplot(common_channel_df, label=fda_name, fill=True, alpha=0.4, ax=axs[0], bw_method='scott', bw_adjust=0.2)
        
        hypercube_df = df[df['fda_name'] == fda_name]['hypercube_time_cost']
        avg_time_cost_hypercube_dict[fda_name] = hypercube_df.mean()
        avg_info_hypercube.append(f'{fda_name}: {avg_time_cost_hypercube_dict[fda_name]:.2f} sec')
        sns.kdeplot(hypercube_df, label=fda_name, fill=True, alpha=0.4, ax=axs[1], bw_method='scott', bw_adjust=0.2)
        
    
    text = "Average Time Cost:\n" + '\n'.join(avg_info_common_channel)
    # Add the text annotation inside the plot
    axs[0].text(0.62, 0.97, text, transform=axs[0].transAxes, fontsize=9, verticalalignment='top',
             bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.4))
    
    axs[0].set_xlim(left=0)
    axs[0].set_xlabel('Time Cost (sec)')
    axs[0].set_ylabel('Density')
    axs[0].legend()
    axs[0].set_title("Common Channel Communication Model")
    axs[0].grid(True, linestyle='--', linewidth=0.5, alpha=0.5)
    axs[0].legend()

    text = "Average Time Cost:\n" + '\n'.join(avg_info_hypercube)
    # Add the text annotation inside the plot
    axs[1].text(0.62, 0.97, text, transform=axs[1].transAxes, fontsize=9, verticalalignment='top',
             bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.4))
    
    axs[1].set_xlim(left=0)
    axs[1].set_xlabel('Time Cost (sec)')
    axs[1].set_ylabel('Density')
    axs[1].legend()
    axs[1].set_title("Hypercube Communication Model")
    axs[1].grid(True, linestyle='--', linewidth=0.5, alpha=0.5)
    axs[1].legend()
    
    plt.tight_layout()
    
    pdf.savefig(fig)
        
    # Close the current figure to prevent it from being displayed in the notebook
    plt.close(fig)
    pdf.close()

## Total Communication cost (in gb) with accuracy (scatter)

### KDE

In [29]:
def kde_communication_cost(df, filename):
    pdf = PdfPages(filename)
    
    plt.figure(figsize=(10, 6))

    avg_communications_dict = {}
    avg_info = []

    for fda_name in fda_names:
        fda_data_df = df[df['fda_name'] == fda_name]['total_communication_gb']
        avg_communications_dict[fda_name] = fda_data_df.mean()
        avg_info.append(f'{fda_name}: {avg_communications_dict[fda_name]:.2f} GB')
        
        # Plotting only the KDE using kdeplot
        sns.kdeplot(fda_data_df, label=fda_name, fill=True, alpha=0.4)

    text = "Average Communication:\n" + '\n'.join(avg_info)
    # Add the text annotation inside the plot
    plt.text(0.62, 0.97, text, transform=plt.gca().transAxes, fontsize=9, verticalalignment='top',
             bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.4))

    plt.xlim(left=0)
    plt.grid(True, linestyle='--', linewidth=0.5, alpha=0.5)
    plt.xlabel('Communication (GB)')
    plt.ylabel('Density')
    plt.legend()
    plt.tight_layout()
    
    pdf.savefig(plt.gcf()) # Save the current figure
        
    # Close the current figure to prevent it from being displayed in the notebook
    plt.close()
    pdf.close()

## Total CPU time (in Seconds) with accuracy

### KDE

In [30]:
def kde_cpu_time_cost(df, filename):
    pdf = PdfPages(filename)
    
    plt.figure(figsize=(10, 6))

    avg_cpu_dict = {}
    avg_info = []

    for fda_name in fda_names:
        fda_data_df = df[df['fda_name'] == fda_name]['cpu_time_cost']
        avg_cpu_dict[fda_name] = fda_data_df.mean()
        avg_info.append(f'{fda_name}: {avg_cpu_dict[fda_name]:.2f} sec')
        
        # Plotting only the KDE using kdeplot
        sns.kdeplot(fda_data_df, label=fda_name, fill=True, alpha=0.4)

    text = "Average CPU cost:\n" + '\n'.join(avg_info)
    # Add the text annotation inside the plot
    plt.text(0.62, 0.97, text, transform=plt.gca().transAxes, fontsize=9, verticalalignment='top',
             bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.4))

    plt.xlim(left=0)
    plt.grid(True, linestyle='--', linewidth=0.5, alpha=0.5)
    plt.xlabel('CPU time cost (sec)')
    plt.ylabel('Count')
    plt.legend()
    plt.tight_layout()
    
    pdf.savefig(plt.gcf()) # Save the current figure
        
    # Close the current figure to prevent it from being displayed in the notebook
    plt.close()
    pdf.close()

## Communication/CPU time - Accuracy

### Scatter

In [31]:
def scatter_time_cost_cpu_ratio(df, filename):
    
    pdf = PdfPages(filename)

    fig, axs = plt.subplots(1, 2, figsize=(20, 6))

    # Plot the data points for each method (fda_name) for single-bus model
    for fda_name in fda_names:
        fda_filtered_data = df[(df['fda_name'] == fda_name)] 
        axs[0].scatter(fda_filtered_data['common_channel_comm_cpu_time_ratio'], fda_filtered_data['accuracy'], label=fda_name)

    axs[0].set_xlabel('(Communication time) / (CPU time)')
    axs[0].set_ylabel('Accuracy')
    axs[0].legend()
    axs[0].set_title("Common Channel Communication Model")

    # Plot the data points for each method (fda_name) for multi-bus model
    for fda_name in fda_names:
        fda_filtered_data = df[(df['fda_name'] == fda_name)] 
        axs[1].scatter(fda_filtered_data['hypercube_comm_cpu_time_ratio'], fda_filtered_data['accuracy'], label=fda_name)

    axs[0].grid(True, linestyle='--', linewidth=0.5, alpha=0.5)
    axs[1].grid(True, linestyle='--', linewidth=0.5, alpha=0.5)   
        
    axs[1].set_xlabel('(Communication time) / (CPU time)')
    axs[1].set_ylabel('Accuracy')
    axs[1].legend()
    axs[1].set_title("Hypercube Communication Model")
    
    axs[0].grid(True, linestyle='--', linewidth=0.5, alpha=0.5)
    axs[1].grid(True, linestyle='--', linewidth=0.5, alpha=0.5)

    plt.tight_layout()
    pdf.savefig(fig)
        
    # Close the current figure to prevent it from being displayed in the notebook
    plt.close(fig)
    pdf.close()

## Different FDA-method runs visualization with lines (per clients)

In [32]:
import matplotlib
from matplotlib import cm

def fda_method_run_line_plot(df, fda_name, filename, limit_x_axis=False):
    
    hybrid_df = df[df.fda_name == fda_name]
    
    batch_size_values = sorted(hybrid_df['batch_size'].unique())
    theta_values = sorted(hybrid_df['theta'].unique())
    
    pdf = PdfPages(filename)

    for batch_size in batch_size_values:
        for theta in theta_values:
            filtered_data = hybrid_df[(hybrid_df['theta'] == theta) &
                                       (hybrid_df['batch_size'] == batch_size)] 

            if filtered_data.empty:
                continue
            
            num_clients_values = sorted(filtered_data['num_clients'].unique())
            
            #print(f"{batch_size} {theta} {filtered_data[['fda_name','accuracy']]}")
                
            # Because num_clients = 2 reaches very good accuracy very early we need to limit the x-axis so we can 
            # visualize the rest of the data more easily. We get the next max time cost and limit x
            # Get the maximum 'common_channel_time_cost' and 'hypercube_time_cost' when num_clients is not 2
            if limit_x_axis:
                # The num clients = 2 cause problems. We put limit 10 after the next in line
                max_common_channel_time_cost = filtered_data[filtered_data['num_clients'] != 2]['common_channel_time_cost'].max()
                max_hypercube_time_cost = filtered_data[filtered_data['num_clients'] != 2]['hypercube_time_cost'].max()


            fig, axs = plt.subplots(1, 2, figsize=(20, 6))

            # Plot each group with a unique color based on num_clients
            for num_clients in num_clients_values:
                data = filtered_data[filtered_data['num_clients'] == num_clients]

                axs[0].plot(data['common_channel_time_cost'], data['accuracy'], color=colors_dict[num_clients], label=num_clients, marker='o', markersize=3)
                axs[1].plot(data['hypercube_time_cost'], data['accuracy'], color=colors_dict[num_clients], label=num_clients, marker='o', markersize=3)
                
            
            axs[0].grid(True, linestyle='--', linewidth=0.5, alpha=0.5)
            axs[1].grid(True, linestyle='--', linewidth=0.5, alpha=0.5)
            
            #axs[0].set_ylim(top=1)
            #axs[1].set_ylim(top=1)

            axs[0].set_xlabel('Time Cost')
            axs[0].set_ylabel('Accuracy')
            axs[0].set_title("Common Channel Communication Model")
            axs[0].legend(title='Num Clients')
            if limit_x_axis:
                axs[0].set_xlim(0, max_common_channel_time_cost+700)  # set x-axis limit

            axs[1].set_xlabel('Time Cost')
            axs[1].set_ylabel('Accuracy')
            axs[1].set_title("Hypercube Communication Model")
            axs[1].legend(title='Num Clients')
            if limit_x_axis:
                axs[1].set_xlim(0, max_hypercube_time_cost+500)  # set x-axis limit

            title = f'Batch Size : {batch_size} , $\Theta$ : {theta}'

            fig.suptitle(title)

            plt.tight_layout()

            pdf.savefig(fig)

            plt.close(fig)

    pdf.close()

## Keep Hyper-parameters fixed and plot for filtered

In [33]:
def fda_methods_batch_size(df, filename):
    pdf = PdfPages(filename)
    
    num_clients_values = sorted(df['num_clients'].unique())
    theta_values = sorted(df['theta'].unique())[1:]

    for num_clients in num_clients_values:
        for theta in theta_values:
            filtered_df = df[(df['theta'] == theta) & (df['num_clients'] == num_clients)] 
            synchronous_data = df[(df['num_clients'] == num_clients) & (df['fda_name'] == 'synchronous')] 
            
            fig, axs = plt.subplots(1, 2, figsize=(20, 6))
            
            for fda_name in fda_names:
                
                fda_data = filtered_df[filtered_df['fda_name'] == fda_name]
                
                if fda_data.empty:
                    continue

                axs[0].plot(fda_data['batch_size'], fda_data['common_channel_time_cost'], marker='o', label=fda_name, markersize=3)
                axs[1].plot(fda_data['batch_size'], fda_data['hypercube_time_cost'], marker='o', label=fda_name, markersize=3)
            
            if not synchronous_data.empty:
                axs[0].plot(synchronous_data['batch_size'], synchronous_data['common_channel_time_cost'], marker='o', label='synchronous', markersize=3)
                axs[1].plot(synchronous_data['batch_size'], synchronous_data['hypercube_time_cost'], marker='o', label='synchronous', markersize=3)

            
            if not axs[0].has_data():
                plt.close(fig)
                continue
            
            x_ticks = [32, 64, 128, 256]
            
            axs[0].set_xticks(x_ticks)
            axs[0].set_xlabel('Batch size')
            axs[0].legend()
            axs[0].set_title("Common Channel Communication Model")
            axs[0].set_ylabel('Time Cost')
            axs[0].grid(True, linestyle='--', linewidth=0.5, alpha=0.5)
            
            axs[1].set_xticks(x_ticks)
            axs[1].set_xlabel('Batch size')
            axs[1].legend()
            axs[1].set_title("Hypercube Communication Model")
            axs[1].set_ylabel('Time Cost')
            axs[1].grid(True, linestyle='--', linewidth=0.5, alpha=0.5)
            
            title = f'Num Clients : {num_clients} , $\Theta$ : {theta}'
            fig.suptitle(title)

            plt.tight_layout()
            pdf.savefig(fig)

            plt.close(fig)
    pdf.close()

In [34]:
def fda_methods_clients(df, filename):
    pdf = PdfPages(filename)
    
    batch_size_values = sorted(df['batch_size'].unique())
    theta_values = sorted(df['theta'].unique())[1:]

    for batch_size in batch_size_values:
        for theta in theta_values:
            
            filtered_df = df[(df['theta'] == theta) & (df['batch_size'] == batch_size)] 
            
            if filtered_df.empty:
                continue
                
            synchronous_data = df[(df['batch_size'] == batch_size) & (df['fda_name'] == 'synchronous')] 
            
            fig, axs = plt.subplots(1, 2, figsize=(20, 6))
            
            for fda_name in fda_names:
                
                fda_data = filtered_df[filtered_df['fda_name'] == fda_name]
                
                if fda_data.empty:
                    continue

                empty = False
                axs[0].plot(fda_data['num_clients'], fda_data['common_channel_time_cost'], marker='o', label=fda_name, markersize=3)
                axs[1].plot(fda_data['num_clients'], fda_data['hypercube_time_cost'], marker='o', label=fda_name, markersize=3)
            
            if not synchronous_data.empty:
                axs[0].plot(synchronous_data['num_clients'], synchronous_data['common_channel_time_cost'], marker='o', label='synchronous', markersize=3)
                axs[1].plot(synchronous_data['num_clients'], synchronous_data['hypercube_time_cost'], marker='o', label='synchronous', markersize=3)

            if not axs[0].has_data():
                plt.close(fig)
                continue
                
            # Set xticks every 5 units based on available values
            min_clients = 5
            max_clients = 60
            
            x_ticks = list(range(min_clients, max_clients + 1, 5))
            
            axs[0].set_xticks(x_ticks)
            axs[0].set_xlabel('Number of clients')
            axs[0].legend()
            axs[0].set_title("Common Channel Communication Model")
            axs[0].set_ylabel('Time Cost')
            axs[0].grid(True, linestyle='--', linewidth=0.5, alpha=0.5)
            
            axs[1].set_xticks(x_ticks)
            axs[1].set_xlabel('Number of clients')
            axs[1].legend()
            axs[1].set_title("Hypercube Communication Model")
            axs[1].set_ylabel('Time Cost')
            axs[1].grid(True, linestyle='--', linewidth=0.5, alpha=0.5)
            
            title = f'Batch Size : {batch_size} , $\Theta$ : {theta}'
            fig.suptitle(title)

            plt.tight_layout()

            pdf.savefig(fig)

            plt.close(fig)
    pdf.close()

# Help-Stat

In [42]:
def explore_top(df, acc_thresh, nn_name, fda_name):
    acceptable_acc_df = df[(df.accuracy > acc_thresh) & (df.nn_name == nn_name)]
    acceptable_acc_df = acceptable_acc_df[acceptable_acc_df['fda_name'] == fda_name]
    idx = acceptable_acc_df.groupby(['fda_name', 'num_clients', 'batch_size', 'theta'])['epoch'].idxmin()
    filtered_acceptable_acc_df = acceptable_acc_df.loc[idx]
    return filtered_acceptable_acc_df[['num_clients', 'batch_size', 'theta', 'total_rounds', 'total_fda_steps', 'epoch', 'total_communication_gb', 'cpu_time_cost', 'common_channel_time_cost']].sort_values(by='common_channel_time_cost')

In [43]:
def mean_epoch_per_method(df, acc_thresh, nn_name, fda_name):
    top_df = explore_top(df, acc_thresh, nn_name, fda_name)
    print(f"Mean epochs : {top_df['epoch'].mean()}")
    print(f"Std epochs : {top_df['epoch'].std()}")
    print(f"Mean rounds : {top_df['total_rounds'].mean()}")
    print(f"Std rounds : {top_df['total_rounds'].std()}")

## Save all those time-cost plots

In [37]:
import os

def time_cost_plots(df, acc_threshold, nn_name, limit_x_axis=False, show_runs=False, params=False, addi_name=''):
    # Filter out based on `acc_threshold`
    acceptable_acc_df = df[(df.accuracy > acc_threshold) & (df.nn_name == nn_name)]
    
    str_thresh = str(acc_threshold).replace('.', '_')  # replace '.'
    
    if not os.path.exists(f"../../metrics/plots/{nn_name}/{str_thresh}"):
        os.makedirs(f"../../metrics/plots/{nn_name}/{str_thresh}")
    
    # 1. Same runs are included
    #scatter_time_cost(acceptable_acc_df, f"../../metrics/plots/{nn_name}/{str_thresh}/nonfiltered_scatter_time_cost.pdf")
    #scatter_time_cost_cpu_ratio(acceptable_acc_df, f"../../metrics/plots/{nn_name}/{str_thresh}/nonfiltered_scatter_time_cost_cpu_ratio.pdf")
    
    if show_runs:
        # Plot the runs of each method. x-axis : time cost, y-axis : accuracy, PER number of clients (lines)
        fda_method_run_line_plot(acceptable_acc_df, 'sketch', f"../../metrics/plots/{nn_name}/{str_thresh}/sketch_run.pdf", limit_x_axis=limit_x_axis)
        fda_method_run_line_plot(acceptable_acc_df, 'naive', f"../../metrics/plots/{nn_name}/{str_thresh}/naive_run.pdf", limit_x_axis=limit_x_axis)
        fda_method_run_line_plot(acceptable_acc_df, 'linear', f"../../metrics/plots/{nn_name}/{str_thresh}/linear_run.pdf", limit_x_axis=limit_x_axis)
        fda_method_run_line_plot(acceptable_acc_df, 'synchronous', f"../../metrics/plots/{nn_name}/{str_thresh}/synchronous_run.pdf", limit_x_axis=limit_x_axis)
    
    # 2. Filter out same runs. We choose the instance which first hits the `acc_threshold`
    idx = acceptable_acc_df.groupby(['fda_name', 'num_clients', 'batch_size', 'theta'])['epoch'].idxmin()
    filtered_acceptable_acc_df = acceptable_acc_df.loc[idx]
    
    # 2. Same runs are NOT included
    kde_time_cost(filtered_acceptable_acc_df, f"../../metrics/plots/{nn_name}/{str_thresh}/kde_time_cost{addi_name}.pdf")
    
    #scatter_time_cost_cpu_ratio(filtered_acceptable_acc_df, f"../../metrics/plots/{nn_name}/{str_thresh}/scatter_time_cost_cpu_ratio.pdf")
    
    #kde_communication_cost(filtered_acceptable_acc_df, f"../../metrics/plots/{nn_name}/{str_thresh}/kde_communication_cost.pdf")
    
    #kde_cpu_time_cost(filtered_acceptable_acc_df, f"../../metrics/plots/{nn_name}/{str_thresh}/kde_cpu_time_cost.pdf")
    
    # Parameters fixed, and plot
    if params:
        fda_methods_batch_size(filtered_acceptable_acc_df, f"../../metrics/plots/{nn_name}/{str_thresh}/fda_methods_batch_size.pdf")
        fda_methods_clients(filtered_acceptable_acc_df, f"../../metrics/plots/{nn_name}/{str_thresh}/fda_methods_clients.pdf")

In [ ]:
time_cost_plots(df, 0.95, 'LeNet-5')

In [ ]:
time_cost_plots(df, 0.955, 'LeNet-5')

In [ ]:
time_cost_plots(df, 0.96, 'LeNet-5')

In [ ]:
time_cost_plots(df, 0.965, 'LeNet-5')

In [ ]:
time_cost_plots(df, 0.97, 'LeNet-5')

In [ ]:
time_cost_plots(df, 0.975, 'LeNet-5')

In [ ]:
time_cost_plots(df, 0.98, 'LeNet-5')

In [ ]:
time_cost_plots(df, 0.985, 'LeNet-5')

In [ ]:
time_cost_plots(df, 0.98, 'AdvancedCNN')

In [ ]:
time_cost_plots(df, 0.985, 'AdvancedCNN')

In [ ]:
time_cost_plots(df, 0.988, 'AdvancedCNN')

In [ ]:
time_cost_plots(df, 0.99, 'AdvancedCNN')

In [ ]:
time_cost_plots(df, 0.993, 'AdvancedCNN')

In [ ]:
time_cost_plots(df, 0.995, 'AdvancedCNN')

In [44]:
pd.set_option('display.max_rows', None)

In [45]:
explore_top(df, 0.995, 'AdvancedCNN', 'synchronous')

,num_clients,batch_size,theta,total_rounds,total_fda_steps,epoch,total_communication_gb,cpu_time_cost,common_channel_time_cost
376378,5,256,0.0,1360,1360,29,141.015789,22.399200,924.900248
345375,15,128,0.0,813,813,26,252.895227,9.747057,1898.031419
356424,10,256,0.0,1758,1758,75,364.567289,28.954260,2653.838743
383361,5,32,0.0,4500,4500,12,466.596360,39.838500,3026.055204
346390,20,128,0.0,961,961,41,398.576980,11.521429,3040.706473
388975,5,64,0.0,4875,4875,26,505.479390,50.334375,3285.402471
354402,5,128,0.0,4969,4969,53,515.226070,59.573341,3357.020186
347394,25,128,0.0,844,844,45,437.563698,10.118716,3370.607914
344410,10,128,0.0,2860,2860,61,593.095818,34.288540,4304.578427
358468,15,256,0.0,1860,1860,119,578.579486,30.634200,4350.694365


In [46]:
explore_top(df, 0.995, 'AdvancedCNN', 'naive')

,num_clients,batch_size,theta,total_rounds,total_fda_steps,epoch,total_communication_gb,cpu_time_cost,common_channel_time_cost
258622,10,32,100.0,28,4313,23,5.806705,38.182989,79.991265
273805,5,32,10.0,119,2250,6,12.338927,19.919250,98.888380
249369,5,256,1.5,138,938,20,14.308974,15.448860,107.026292
293654,5,64,1.5,159,938,5,16.486423,9.684850,115.197960
274258,5,32,15.0,132,3375,9,13.686894,29.878875,117.474997
259158,10,32,15.0,74,1688,9,15.345903,14.943864,125.434368
274465,5,32,30.0,118,6000,16,12.235313,53.118000,131.424006
261268,15,32,30.0,56,2375,19,17.419740,21.025875,151.093267
259470,10,32,30.0,82,3938,21,17.005003,34.863114,157.299133
264866,30,32,100.0,25,4188,67,15.553715,37.076364,157.358423


In [47]:
explore_top(df, 0.985, 'LeNet-5', 'synchronous')

,num_clients,batch_size,theta,total_rounds,total_fda_steps,epoch,total_communication_gb,cpu_time_cost,common_channel_time_cost
928479,5,256,0.0,1407,1407,30,3.472814,13.026006,35.252014
913491,10,256,0.0,985,985,42,4.862433,9.119130,44.128646
911974,5,128,0.0,2344,2344,25,5.785555,18.798880,55.826429
938463,5,64,0.0,2625,2625,14,6.479130,19.711125,61.177557
904482,10,128,0.0,1547,1547,33,7.636735,12.406940,67.391429
915029,15,256,0.0,1250,1250,80,9.255900,11.572500,80.683220
905245,15,128,0.0,1438,1438,46,10.647987,11.532760,91.037732
934474,10,64,0.0,2344,2344,25,11.571109,17.601096,100.913082
916554,20,256,0.0,1231,1231,105,12.153614,11.396598,103.764063
906005,20,128,0.0,1313,1313,56,12.963196,10.530260,109.050553


In [48]:
explore_top(df, 0.985, 'LeNet-5', 'naive')

,num_clients,batch_size,theta,total_rounds,total_fda_steps,epoch,total_communication_gb,cpu_time_cost,common_channel_time_cost
694581,5,256,1.0,41,1500,32,0.101228,13.887000,14.534858
617611,10,256,1.5,37,1454,62,0.182708,13.461132,14.776629
615610,10,256,0.5,78,1430,61,0.385103,13.238940,16.011679
572601,15,128,1.0,54,1625,52,0.399952,13.032500,16.018811
622645,15,256,1.0,56,1500,96,0.414754,13.887000,16.983832
638696,25,256,1.0,55,1379,147,0.678904,12.766782,17.980764
571602,15,128,0.5,85,1657,53,0.629501,13.289140,17.989411
608572,5,128,1.0,53,2157,23,0.130860,17.299140,18.136643
625661,15,256,1.5,42,1750,112,0.311103,16.201500,18.524404
620645,15,256,0.5,84,1500,96,0.622086,13.887000,18.531912


In [ ]:
mean_epoch_per_method(df, 0.98, 'LeNet-5', 'synchronous')

In [ ]:
mean_epoch_per_method(df, 0.995, 'AdvancedCNN', 'naive')

In [ ]:
mean_epoch_per_method(df, 0.995, 'AdvancedCNN', 'synchronous')